<a href="https://colab.research.google.com/github/vinayk19/Assignment/blob/master/13_2_rnn_classificaiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/My Drive/AI/basic

/content/drive/My Drive/AI/basic


In [2]:
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
# from torch.utils.data import DataLoader
# from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import csv
import gzip
# from name_dataset import NameDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [3]:
pwd

'/content/drive/My Drive/AI/basic'

In [4]:
class NameDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self, is_train_set=False):
        filename = '/content/drive/My Drive/AI/basic/data/names_train/names_train.csv.gz' if is_train_set else '/content/drive/My Drive/AI/basic/data/names_train/names_test.csv.gz'
        with gzip.open(filename, "rt") as f:
            reader = csv.reader(f)
            rows = list(reader)

        self.names = [row[0] for row in rows]
        self.countries = [row[1] for row in rows]
        self.len = len(self.countries)

        self.country_list = list(sorted(set(self.countries))) # learning it provides the complete list of the country

    def __getitem__(self, index):
        return self.names[index], self.countries[index]

    def __len__(self):
        return self.len

    def get_countries(self):
        return self.country_list

    def get_country(self, id):
        return self.country_list[id] #it will provide contry list (after wbing shorted) it will provide the ID

    def get_country_id(self, country):
        return self.country_list.index(country) #learnign to get the index of country ( secong collumn of csv file)

In [5]:
# Test the loader
if __name__ == "__main__":
    dataset = NameDataset(False)
    print("get_country",dataset.get_countries())
    print(dataset.get_country(4))
    print(dataset.get_country_id('English'))

    train_loader = DataLoader(dataset=dataset,
                              batch_size=10,
                              shuffle=True)

    print(len(train_loader.dataset))
    # for epoch in range(2):
    #     for i, (names, countries) in enumerate(train_loader):
    #         # Run your training process
    #         print(epoch, i, "names", names, "countries", countries)

get_country ['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']
English
4
6700


In [6]:
# Parameters and DataLoaders
HIDDEN_SIZE = 100
N_LAYERS = 2
BATCH_SIZE = 256
N_EPOCHS = 3

test_dataset = NameDataset(is_train_set=False)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=True)


train_dataset = NameDataset(is_train_set=True)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True)

N_COUNTRIES = len(train_dataset.get_countries())
print(N_COUNTRIES, "countries")
N_CHARS = 128  # ASCII

18 countries


In [7]:
# Some utility functions
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def create_variable(tensor):
    # Do cuda() before wrapping with variable
    if torch.cuda.is_available():
        return Variable(tensor.cuda())
    else:
        return Variable(tensor)

In [8]:
# pad sequences and sort the tensor
def pad_sequences(vectorized_seqs, seq_lengths, countries):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)

    # Sort tensors by their length @can we ignore shortung ?
    seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]

    # Also sort the target (countries) in the same order
    target = countries2tensor(countries)
    if len(countries):
        target = target[perm_idx]
        
       # Return variables
    # DataParallel requires everything to be a Variable
    return create_variable(seq_tensor), \
        create_variable(seq_lengths), \
        create_variable(target)

In [9]:
# Create necessary variables, lengths, and target
def make_variables(names, countries):
    print("\n in input type", type(names), "input Contry ", type(countries))
    print("\nin input word", names, "\ninput country", countries)
    sequence_and_length = [str2ascii_arr(name) for name in names]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
    print("\n in vectorized seqs type", type(vectorized_seqs), "\nSeq lengths ", type(seq_lengths))
    print("\n in vectorized seqs type", vectorized_seqs, "\nSeq lengths ", seq_lengths, "\n outcountry", countries, "\ncountry type", type(countries))

    return pad_sequences(vectorized_seqs, seq_lengths, countries)

def str2ascii_arr(msg):
    arr = [ord(c) for c in msg]
    return arr, len(arr)

def countries2tensor(countries):
    country_ids = [train_dataset.get_country_id(
        country) for country in countries]
    return torch.LongTensor(country_ids)

In [17]:
class RNNClassifier(nn.Module):
    # Our model classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRIES, N_LAYERS)

    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True): # output_size is for linear layer output ie N_country
        super(RNNClassifier, self).__init__()
        print("inside model type", type(input_size), type(hidden_size), type(output_size), type(n_layers))
        print("inside model vlaue", input_size, hidden_size, output_size, n_layers)
        self.hidden_size = hidden_size # it is like output in RNN
        self.n_layers = n_layers
        self.n_directions = int(bidirectional) + 1

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, seq_lengths): #learning both ipput are Variable and torch.Tensor
        # Note: we run this all at once (over the whole input sequence)
        # input shape: B x S (input size)
        # transpose to make S(sequence) x B (batch)
        print("\nforward input type", type(input), "\nseq_lenght type", type(seq_lengths))
        print("\nforwward input word", input.shape, "\nseq_lenght", seq_lengths.shape)
        input = input.t()
        batch_size = input.size(1)

        # Make a hidden
        hidden = self._init_hidden(batch_size)

        print("\n1embedding input", input.shape)
        # Embedding S x B -> S x B x I (embedding size)
        embedded = self.embedding(input)
        print("\n2embedding", embedded.shape)

        print("\n2seq_lengths.data.cpu().numpy()", seq_lengths.data.cpu().numpy())
        print("\n2seq_lengths.data.cpu().numpy()", type(seq_lengths.data.cpu().numpy()))
        print("\n2seq_lengths.data.cpu().numpy()", seq_lengths.data.cpu().numpy().shape)

        # print("\n2seq_lengths.data.cpu().numpy()", seq_lengths.data.cpu().numpy(), type(seq_lengths.data.cpu().numpy(), seq_lengths.data.cpu().numpy().shape))
        # Pack them up nicely # what is pack padded sequence
        gru_input = pack_padded_sequence(embedded, seq_lengths.data.cpu().numpy()) # LENGTH SHOULD BE TENSOR BUT IT IS NUMPY
        print("\n3Gru input", type(gru_input), gru_input)

        # To compact weights again call flatten_parameters().
        self.gru.flatten_parameters() #learning what is in flattening 
        output, hidden = self.gru(gru_input, hidden)  #gru_iput is not tensor it is rnn_class

        # Use the last layer output as FC's input
        # No need to unpack, since we are going to use hidden
        fc_output = self.fc(hidden[-1])
        return fc_output

    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                             batch_size, self.hidden_size)
        return create_variable(hidden)

In [11]:
# Train cycle
def train():
    total_loss = 0

    for i, (names, countries) in enumerate(train_loader, 1):
        input, seq_lengths, target = make_variables(names, countries) #learning out are tensor and careate as Variable as well.
        output = classifier(input, seq_lengths)
        print("\n output", output.shape, type(output), type(target))
        loss = criterion(output, target)
        print("\nloss value type is ", type(loss.data), type(loss.item()))
        print("\nloss value is ", loss.data, loss.item())
        total_loss += loss.data[0]
        # total_loss += loss.item()

        classifier.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                time_since(start), epoch,  i *
                len(names), len(train_loader.dataset),
                100. * i * len(names) / len(train_loader.dataset),
                total_loss / i * len(names)))

    return total_loss

In [12]:
# Testing cycle
def test(name=None):
    # Predict for a given name
    if name:
        input, seq_lengths, target = make_variables([name], [])
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        country_id = pred.cpu().numpy()[0][0]
        print(name, "is", train_dataset.get_country(country_id))
        return

    print("evaluating trained model ...")
    correct = 0
    train_data_size = len(test_loader.dataset)

    for names, countries in test_loader:
        input, seq_lengths, target = make_variables(names, countries)
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))

In [18]:
if __name__ == '__main__':

    print("type", type(N_CHARS), type(HIDDEN_SIZE), type(N_COUNTRIES), type(N_LAYERS))
    print("value", N_CHARS, HIDDEN_SIZE, N_COUNTRIES, N_LAYERS)
    classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRIES, N_LAYERS)
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        # dim = 0 [33, xxx] -> [11, ...], [11, ...], [11, ...] on 3 GPUs
        classifier = nn.DataParallel(classifier)

    if torch.cuda.is_available():
        classifier.cuda()

    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    start = time.time()
    print("Training for %d epochs..." % N_EPOCHS)
    for epoch in range(1, N_EPOCHS + 1):
        # Train cycle
        train()

        # Testing
        test()

        # Testing several samples
        test("Sung")
        test("Jungwoo")
        test("Soojin")
        test("Nako")


type <class 'int'> <class 'int'> <class 'int'> <class 'int'>
value 128 100 18 2
inside model type <class 'int'> <class 'int'> <class 'int'> <class 'int'>
inside model vlaue 128 100 18 2
Training for 3 epochs...

 in input type <class 'tuple'> input Contry  <class 'tuple'>

in input word ('Masih', 'Vikhrov', 'Paramoshkin', 'Asker', 'Adderley', 'Mahrov', 'Whiteley', 'Tulchinsky', 'Gerges', 'Gaber', 'Homentovsky', 'Komine', 'Belchenkov', 'Maciomhair', 'Anikin', 'Muntyan', 'Guzun', 'Dovgalevsky', 'Abboud', 'Abletsov', 'Lavigne', 'Dikusar', 'Mischenko', 'Bahtchivandzhi', 'Chepurenko', 'Shadhan', 'Nahas', 'Pendyurin', 'BeknazarYuzbashev', 'Timpe', 'Jankevich', 'Qureshi', 'Pechernikov', 'Estlick', 'Shamoon', 'Balagul', 'Ha', 'Tupikhin', 'Zimnitsky', 'Atopov', 'Mayer', 'Talkov', 'Sasada', 'Molevich', 'Divoky', 'Christie', 'Varaksin', 'Grosu', 'Najjar', 'Pyschev', 'Vilken', 'Cardozo', 'Zhivlyuk', 'Gauntlett', 'Jakhaev', 'Golovanov', 'Zavorokhin', 'Latham', 'Lake', 'Hlyupin', 'Minchenkov', 'Ross

IndexError: ignored